# CO2 Concentration in the Atmosphere

In [ ]:
#matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt

#1 The data file weekly_in_situ_co2_mlo.csv is downloaded from https://scrippsco2.ucsd.edu/data/atmospheric_co2/mlo.html. Download date 03/01/2025 14:29.
